In [1]:
# 순환 신경망

# 데이터 불러오기
from tensorflow import keras
(train_input, train_target), (test_input, test_target)=\
  keras.datasets.imdb.load_data(num_words=300)
  # 상위 빈출 단어 300개만 이용
# 문자열로 된 리뷰가 숫자로 치환된 채 저장돼 있음 | 2: 상위 빈출 단어 300개에 미포함된 단어는 모두 2로 처리돼 있음

In [2]:
# 훈련-검증 셋 구분
from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target)


# 모든 리뷰 데이터 동일 규격 통일 필요
from keras.preprocessing.sequence import pad_sequences
train_seq = pad_sequences(train_input, maxlen=100)
  # 모든 데이터 최대 길이 100단어로 쪼갬 -> 학습시킬 때 통일된 규격의 데이터 넣어야 하기 때문
  # maxlen=: 입력 길이보다 적은 단어는 패딩 넣고, 초과 단어는 입력 길이만큼만 자름
    # 초과 단어: 뒤부터 100개 단어만 남김 -> 문장에서 중요한 건 뒤에 있다 설계돼서
train_seq[0]

array([  2,  56,   5,   2,  18,  68,  38,   2,   2,  63,  26,   2,   8,
       156,   2, 262,   2,   2,  37, 115,  79,  68, 194,   2,  10,  10,
         2,   4, 206,   4,   2,   9,  38,  38,  33, 118,   5,   4,   2,
         2,   9,   2, 207, 110,  53, 147, 267,   2,  33,   4,   2,   2,
         5, 148,  71, 184,   2, 267,  10,  10,   2,   2,   8,  72,   2,
        14,   2,   7,   2,  33,  32,   2,  48,  12,  71,   4,  64,  22,
        11,   2,  25, 131,  62, 181,   8,   2,  12,  18,   4,   2,   7,
       129,   2,   2,   2,  81,  24, 106,  14, 155], dtype=int32)

In [3]:
val_seq = pad_sequences(val_input, maxlen=100)
val_seq.shape


(6250, 100)

In [4]:
# 모델 만들기
model = keras.Sequential() # 모델 선언
model.add(keras.layers.SimpleRNN(8, input_shape=(100, 300)))
  # 8개의 셀 | 100개의 토큰 들어오는데, 이는 300개의 숫자 범위 내에 들어 있단 것
  # SimpleRNN 기본값: tanh 함수 사용
model.add(keras.layers.Dense(1, activation='sigmoid')) # 뉴런 1개, 0/1로만 구분하면 되니까 시그모이드 사용

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
train_oh = keras.utils.to_categorical(train_seq)
  # 단어 하나 표현 위해 300개의 칸 더미변수 처럼 만든 것 -> 숫자 의미 없애기 위해 한 것
train_oh.shape

(18750, 100, 300)

In [6]:
val_oh = keras.utils.to_categorical(val_seq)
val_oh.shape

(6250, 100, 300)

In [ ]:
# 학습
import tensorflow as tf
from tensorflow import keras

rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer=rmsprop,
              loss='binary_crossentropy', # 긍부정 이진 분류하기에 사용
              metrics=['accuracy'])
es = keras.callbacks.EarlyStopping(patience=3)
result = model.fit(train_oh, train_target, epochs=30,
                   validation_data=(val_oh, val_target),
                   callbacks=es)

In [8]:
# 새 모델 만들기
model2 = keras.Sequential()
model2.add(keras.layers.Embedding(300, 16, input_length=100))
# 16개의 실수들의 집합으로 만들겠단 것, 하나의 리뷰당 100개의 길이 가진 단어 총 300개 들어 있단 것 -> 이를 16개 숫자의 집합으로 만들기
model2.add(keras.layers.SimpleRNN(8))
model2.add(keras.layers.Dense(1, activation='sigmoid'))

# 컴파일
model2.compile(loss='binary_crossentropy',
               metrics=['accuracy'])
es = keras.callbacks.EarlyStopping(patience=2)

# 학습
model2.fit(train_seq, train_target, epochs=100,
           validation_data=(val_seq, val_target),
           callbacks=[es])

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


586/586 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.5821 - loss: 0.6656 - val_accuracy: 0.6750 - val_loss: 0.6069
Epoch 2/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.7084 - loss: 0.5773 - val_accuracy: 0.7286 - val_loss: 0.5478
Epoch 3/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.7355 - loss: 0.5367 - val_accuracy: 0.7424 - val_loss: 0.5319
Epoch 4/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.7494 - loss: 0.5257 - val_accuracy: 0.7474 - val_loss: 0.5291
Epoch 5/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.7568 - loss: 0.5073 - val_accuracy: 0.7395 - val_loss: 0.5282
Epoch 6/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - accuracy: 0.7513 - loss: 0.5110 - val_accuracy: 0.7458 - val_loss: 0.5230
Epoch 7/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.7563 - loss: 0.5110 - val_accuracy: 0.7461 - val_loss: 0.5237
Epoch 8/100
586/586 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.7608 - loss: 0.5065 - val_